# Single image analysis

* This notebook is for analysis of analysis of single images taken at, say, 1 sun conditions at open and short circuit for example, using the PL imaging setup
* Assumes pre processing has already been done
    
### Some instructions: 
* Please run each cell in order, following the instructions
    * Press cntrl+enter to run a cell
* When prompted enter information into the feilds and press enter
* If you go back and re run a cell, that's fine but you have to run every cell after that again, and in order

This might break if all these files are not there! If you want to edit this code definately go ahead, a bunch of the functions live in seperate .py files, BUT please make a copy!!!

In [ ]:
from external_imports import *
from image_process import *
from general_data_process import *
import scipy.io

from intensity_dependant import *
from voltage_dependant import *

# Plot parameters
import matplotlib
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"

## Pull images from intensity dependant
* Run this if what you have intensity dependant data and you want to pull 1 sun images interpolated form that. 
* If you just took the single images yourself skip this

In [ ]:
# Not finished this yet

### Core process: Finding the folders/sample names
* Enter datapath and save path
* You can comment out the approriate parts depending on if they are all the same bandgap

In [ ]:
datapath = r""
savepath = r""
if not os.path.isdir(savepath):
    os.makedirs(savepath)

path_db = path_process(datapath)

bandgaps = {}
for key in path_db.keys():
    if key.lower() != 'white':
#         Different bandgaps (comment/ uncomment as needed):
        
#         string = "What's the bandgap of "+key+" in eV?: "
#         bandgap = float(input(string))
#         bandgaps[key] = bandgap

#         Same bandgaps (comment/ uncomment as needed):
        bandgaps[key] = 1.6 # change this

### Core process: PLQEs

In [ ]:
whiteparamsfile = f"{datapath}/white_params.csv"  # defines location of white ref parameter file

for key in path_db.keys():
    bandgap = bandgaps[key]
    for pix in path_db[key]: 
        whitefilename = f"{datapath}/{key}/{pix}/white.npy"
        for _,biases,_ in os.walk(f"{datapath}/{key}/{pix}"):
            break
        
        # save PLQEs for oc and sc
        for bias in biases:
            datapath_i = f"{datapath}/{key}/{pix}/{bias}"
            savepath_i = f"{savepath}/{key}/{pix}"
            save_PLQE(datapath_i, savepath_i, whitefilename, whiteparamsfile, bandgap, 
                      flux1sun=1, savename=bias)

* If you have externally measured PLQEs (eg from integrating sphere), run this:

In [ ]:
'''
expecting: in supstrate folder:
0012346
|____PLQE_EXT
     |____bias
          |____plqe_ext.txt
'''

external_PLQE_averager(datapath, savepath)

* Flattened csv list (averaged across substrate)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    with open(f"{savepath}/{key}_PLQE_list.csv",'w',newline='') as file:
        writer = csv.writer(file)
    
        for pix in path_db[key]:
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/PLQE_{bias}"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Flattened csv list (per pixel)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        with open(f"{savepath}/{key}_{pix}_PLQE_list.csv",'w',newline='') as file:
            writer = csv.writer(file)
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/PLQE_{bias}"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Plots (Change code to change scales, cmaps, etc)

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        for bias in biases:
            path_i = f"{savepath}/{key}/{pix}/PLQE_{bias}"
            savepath_i = f"{savepath}/Plots/PLQE"
            if not os.path.isdir(savepath_i):
                os.makedirs(savepath_i)
            filename = find_npy(path_i)[0]

            savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
            num_sun = float(filename.split('_')[1])

            fig, ax = plt.subplots()
            imarr = np.load(f"{path_i}/{filename}")
            im = ax.imshow(imarr*100, cmap="inferno", vmin=np.mean(imarr*100)-3*np.std(imarr*100),
                                                      vmax=np.mean(imarr*100)+3*np.std(imarr*100))

            ax.set_title(f'num_sun = {num_sun}')
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('PLQE (%)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{savepath_i}/{savefile_name}", dpi=300)
            plt.close('all')

### QFLS
* Produces maps of QFLS
* Requires only an OC image

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:  
        for bias in biases:
            path_i = f"{savepath}/{key}/{pix}"
            bandgap = bandgaps[key]
            voc_rad0 = vocradf(bandgap)

            intsweep_QFLS(path_i, voc_rad0, bias)

* Flattened csv list (averaged across substrate)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    with open(f"{savepath}/{key}_QFLS_list.csv",'w',newline='') as file:
        writer = csv.writer(file)
    
        for pix in path_db[key]:
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/QFLS_int_oc"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Flattened csv list (per pixel)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        with open(f"{savepath}/{key}_{pix}_QFLS_list.csv",'w',newline='') as file:
            writer = csv.writer(file)
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/QFLS_int_oc"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Plots:

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        for bias in biases:
            path_i = f"{savepath}/{key}/{pix}/QFLS_int_oc"
            savepath_i = f"{savepath}/Plots/QFLS"
            if not os.path.isdir(savepath_i):
                os.makedirs(savepath_i)

            filename = find_npy(path_i)[0]

            savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
            fig, ax = plt.subplots()

            num_suns = float(filename.split('_')[1])
            jbyjsc = 1-num_suns

            imarr = np.load(f"{path_i}/{filename}")
            im = ax.imshow(imarr, cmap="inferno", vmin=1.22, vmax=1.31)
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('V$_{oc}$ (V)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{savepath_i}/{savefile_name}", dpi=300)
            plt.close('all')


### Collection efficiency
* Plots colection efficiency :
($\frac{PL_{oc} - PL_{sc}}{PL_{oc}}$)
* Draws from PLQE_oc and PLQE_sc folder, so that needs to be present

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        for bias in biases:
            path_i = f"{savepath}/{key}/{pix}"
            oc_filename = find_npy(f"{path_i}/PLQE_oc")[0]
            sc_filename = find_npy(f"{path_i}/PLQE_sc")[0]
                        
            im_oc = np.load(f"{path_i}/PLQE_oc/{oc_filename}")
            im_sc = np.load(f"{path_i}/PLQE_sc/{sc_filename}")
            col_eff = (im_oc - im_sc) / im_oc
            if not os.path.isdir(f"{path_i}/col_eff"):
                os.makedirs(f"{path_i}/col_eff")
                filename = find_npy(f"{path_i}/PLQE_oc")[0]
            np.save(f"{path_i}/col_eff/{filename}", col_eff)

* Flattened csv list (averaged across substrate)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    with open(f"{savepath}/{key}_coleff_list.csv",'w',newline='') as file:
        writer = csv.writer(file)
    
        for pix in path_db[key]:
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/col_eff"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Flattened csv list (per pixel)
    * For now, just for single images taken

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        with open(f"{savepath}/{key}_{pix}_coleff_list.csv",'w',newline='') as file:
            writer = csv.writer(file)
            for bias in biases:
                path_i = f"{savepath}/{key}/{pix}/col_eff"
                list_i = np.ravel(np.load(f"{path_i}/{find_npy(path_i)[0]}"))
                for i in list_i:
                    writer.writerow([i])

* Plots:

In [ ]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:
        for bias in biases:
            path_i = f"{savepath}/{key}/{pix}/col_eff"
            savepath_i = f"{savepath}/Plots/col_eff"
            if not os.path.isdir(savepath_i):
                os.makedirs(savepath_i)

            filename = find_npy(path_i)[0]

            savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
            fig, ax = plt.subplots()

            num_suns = float(filename.split('_')[1])
            jbyjsc = 1-num_suns

            imarr = np.load(f"{path_i}/{filename}")
            im = ax.imshow(imarr*100, cmap="inferno", vmin=0, vmax=100)
            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('Collection efficiency (%)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{savepath_i}/{savefile_name}", dpi=300)
            plt.close('all')
